In [ ]:
import cv2
import os
import numpy as np
from keras_facenet import FaceNet
from sklearn.metrics.pairwise import cosine_similarity

# Load FaceNet model
embedder = FaceNet()

# Load face detector (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# ==== Functions ====
def load_image(path):
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (160, 160))
    return img_resized

def get_embedding(img):
    return embedder.embeddings([img])[0]

def build_database(dataset_path):
    database = {}
    for person in os.listdir(dataset_path):
        person_folder = os.path.join(dataset_path, person)
        if not os.path.isdir(person_folder):
            continue
        embeddings = []
        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            try:
                img = load_image(img_path)
                embedding = get_embedding(img)
                embeddings.append(embedding)
            except Exception as e:
                print(f"Lỗi khi xử lý {img_path}: {e}")
        if embeddings:
            database[person] = np.mean(embeddings, axis=0)
    return database

def recognize_face(face_img, database, threshold=0.6):
    try:
        resized = cv2.resize(face_img, (160, 160))
        rgb = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
        emb = get_embedding(rgb)
    except:
        return "Error"
    best_match = "Unknown"
    best_score = -1
    for name, db_emb in database.items():
        score = cosine_similarity([emb], [db_emb])[0][0]
        if score > best_score:
            best_score = score
            best_match = name
    if best_score < threshold:
        return "Unknown"
    return best_match

# ==== MAIN ====
if __name__ == "__main__":
    DATASET_DIR = "E:/face-detec/dataset/train"
    VIDEO_PATH = "E:/face-detec/Trump, Zelensky meeting FULL fiery exchange at White House - FOX 26 Houston (720p, h264).mp4"  # 0: webcam hoặc thay bằng 'your_video.mp4'
    OUTPUT_VIDEO = "E:/face-detec\output\output_facenet.mp4"  # File video xuất ra

    print("Đang tạo database khuôn mặt...")
    database = build_database(DATASET_DIR)

    print("Mở video...")
    cap = cv2.VideoCapture(VIDEO_PATH)

    # Lấy thông tin video (frame size, FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) if cap.get(cv2.CAP_PROP_FPS) > 0 else 20

    # Khởi tạo VideoWriter để lưu video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # codec cho .avi
    out = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            name = recognize_face(face_img, database)

            # Vẽ khung và tên
            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        # Hiển thị real-time
        cv2.imshow("Face Recognition (press Q to quit)", frame)

        # Ghi frame vào file video
        out.write(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    print(f"Video đã lưu tại: {OUTPUT_VIDEO}")



Đang tạo database khuôn mặt...
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━

In [ ]:
import cv2
import os
import numpy as np
from keras_facenet import FaceNet
from sklearn.metrics.pairwise import cosine_similarity

# Load FaceNet model
embedder = FaceNet()

# Load face detector (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# ==== Functions ====
def load_image(path):
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (160, 160))
    return img_resized

def get_embedding(img):
    return embedder.embeddings([img])[0]

def build_database(dataset_path):
    database = {}
    for person in os.listdir(dataset_path):
        person_folder = os.path.join(dataset_path, person)
        if not os.path.isdir(person_folder):
            continue
        embeddings = []
        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            try:
                img = load_image(img_path)
                embedding = get_embedding(img)
                embeddings.append(embedding)
            except Exception as e:
                print(f"Lỗi khi xử lý {img_path}: {e}")
        if embeddings:
            database[person] = np.mean(embeddings, axis=0)
    return database

def recognize_face(face_img, database, threshold=0.6):
    try:
        resized = cv2.resize(face_img, (160, 160))
        rgb = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
        emb = get_embedding(rgb)
    except:
        return "Error"
    best_match = "Unknown"
    best_score = -1
    for name, db_emb in database.items():
        score = cosine_similarity([emb], [db_emb])[0][0]
        if score > best_score:
            best_score = score
            best_match = name
    if best_score < threshold:
        return "Unknown"
    return best_match

# --- Nhận diện qua webcam ---
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = detector.detect_faces(frame)
    for res in results:
        x, y, w, h = res['box']
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (160, 160))
        face = face.astype('float32') / 255.0
        embedding = embedder.embeddings([face])[0]
        name = recognize_face(embedding)

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

    cv2.imshow("Face Recognition - FaceNet", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

In [1]:
import cv2
import os
import numpy as np
from keras_facenet import FaceNet
from sklearn.metrics.pairwise import cosine_similarity

# Load FaceNet model
embedder = FaceNet()

# Load Haar Cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# ==== Functions ====
def load_image(path):
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (160, 160))
    return img_resized

def get_embedding(img):
    return embedder.embeddings([img])[0]

def build_database(dataset_path):
    database = {}
    for person in os.listdir(dataset_path):
        person_folder = os.path.join(dataset_path, person)
        if not os.path.isdir(person_folder):
            continue
        embeddings = []
        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            try:
                img = load_image(img_path)
                embedding = get_embedding(img)
                embeddings.append(embedding)
            except Exception as e:
                print(f"Lỗi khi xử lý {img_path}: {e}")
        if embeddings:
            database[person] = np.mean(embeddings, axis=0)
    return database

def recognize_face(face_img, database, threshold=0.6):
    try:
        resized = cv2.resize(face_img, (160, 160))
        rgb = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
        emb = get_embedding(rgb)
    except:
        return "Error"
    best_match = "Unknown"
    best_score = -1
    for name, db_emb in database.items():
        score = cosine_similarity([emb], [db_emb])[0][0]
        if score > best_score:
            best_score = score
            best_match = name
    if best_score < threshold:
        return "Unknown"
    return best_match

# ==== MAIN ====
if __name__ == "__main__":
    DATASET_DIR = "E:/face-detec/dataset/train"  # Cập nhật đường dẫn phù hợp
    print("Đang xây dựng database khuôn mặt...")
    database = build_database(DATASET_DIR)

    print("Bắt đầu mở webcam...")
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            name = recognize_face(face_img, database)

            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        cv2.imshow("Face Recognition - FaceNet", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



Đang xây dựng database khuôn mặt...
1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 402ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 